In [1603]:
#Proyecto Shark Attack

In [1604]:
import pandas as pd

In [1605]:
#Exportamos la base de dato y creamos un duplicado sobre el que vamos a trabajar
data = pd.read_csv('GSAF2.csv')
data1 = data

In [1606]:
# Comprobamos la cantidad de valores nulos por columnas
null_cols = data1.isnull().sum()
null_cols[null_cols > 0]

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

In [1607]:
# Eliminamos las columnas que no vamos a utilizar
data1 = data1.drop(['Unnamed: 22','Unnamed: 23','Case Number','Case Number.1','Name','Case Number.2','Investigator or Source','pdf','href formula','href','original order','Injury','Type','Species '], axis=1)

In [1608]:
#Renombramos algunos nombres de columnas y las ordenamos
data1 = data1.rename(index=str, columns={"Sex ": "Sex", "Time": "Hour", "Date": "Month"})
column_order = ['Year','Month','Hour','Country','Area','Location','Sex','Age','Activity','Fatal (Y/N)']
data1 = data1[column_order]
data1.head(3)




,Year,Month,Hour,Country,Area,Location,Sex,Age,Activity,Fatal (Y/N)
0,2016,18-Sep-16,13h00,USA,Florida,"New Smyrna Beach, Volusia County",M,16,Surfing,N
1,2016,18-Sep-16,11h00,USA,Florida,"New Smyrna Beach, Volusia County",M,36,Surfing,N
2,2016,18-Sep-16,10h43,USA,Florida,"New Smyrna Beach, Volusia County",M,43,Surfing,N


In [1609]:
#Eliminamos todos los datos anteriores a 1900
years = data1["Year"]
drop_cols = list(years[years < 1900].index)
data1 = data1.drop(drop_cols, axis=0)

# Ordenamos la tabla por año
data1.sort_values(['Year'], ascending = 0).head()

,Year,Month,Hour,Country,Area,Location,Sex,Age,Activity,Fatal (Y/N)
0,2016,18-Sep-16,13h00,USA,Florida,"New Smyrna Beach, Volusia County",M,16,Surfing,N
52,2016,05-Jun-16,11h30,AUSTRALIA,Western Australia,Mindarie,F,60,Diving,Y
76,2016,31-Mar-16,11h00,USA,Hawaii,"Olowalu, Maui",F,46,Snorkeling,N
75,2016,07-Apr-16,NaN,USA,Florida,"Corners Beach, Jupiter, Palm Beach County",M,NaN,SUP,N
74,2016,07-Apr-16,NaN,USA,Florida,"Florida Keys, Monroe County",M,34,Fishing,N


In [1610]:
null_cols1 = data1.isnull().sum()
null_cols1[null_cols1 > 0]

Hour           2633
Country          22
Area            295
Location        363
Sex             505
Age            2103
Activity        426
Fatal (Y/N)       9
dtype: int64

In [1611]:
# Eliminamos las filas que tenían algún valor de las columnas Country, Fatal, Area o Location nulos
data1 = data1.dropna(subset=['Country','Fatal (Y/N)','Area','Location'], axis=0)


In [1612]:
null_cols1 = data1.isnull().sum()
null_cols1[null_cols1 > 0]

Hour        2200
Sex          421
Age         1770
Activity     352
dtype: int64

In [1613]:
#Cambiamos los datos nulos a Unknokn
data1[['Activity','Sex','Age','Hour']] = data1[['Activity','Sex','Age','Hour']].fillna('Unknown')

In [1614]:
#Comprobamos que no queda ningún nulo
null_cols1 = data1.isnull().sum()
null_cols1[null_cols1 > 0]

Series([], dtype: int64)

In [1615]:
#Nos quedamos solo con el mes del ataque 
def fecha(*lab):
    for x in lab:
        data1.loc[data1["Month"].str.contains(x), 'Month'] = x
fecha('Jan','Mar','Feb','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec')

def april(*lab):
    for x in lab:
        data1.loc[data1["Month"].str.contains(x), 'Month'] = 'Apr'
april('Ap')

def nomon(*lab):
    for x in lab:
        data1.loc[data1["Month"].str.startswith(x), 'Month'] = 'Unknown'
nomon('1','2','S','R','C','F','W','E','L')

datavalue = data1[["Month"]].stack().value_counts().index.tolist()
data1["Month"].value_counts()

Unknown    970
Jul        525
Aug        488
Jan        421
Jun        385
Oct        357
Apr        353
Dec        352
Mar        330
Nov        319
May        295
Name: Month, dtype: int64

In [1616]:
datavalue = data1[["Hour"]].stack().value_counts().index.tolist()
display(len(datavalue))

data1["Hour"] = data1["Hour"].str.lower()
def time(*lab):
    for x in lab:
        data1.loc[data1["Hour"].str.startswith(x), 'Hour'] = x + "h"
time('00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19',"20","21","22","23")

letras =['<','>','2 ','8','3','� ','500','"',' ','-','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
def notime(lab):
    for x in lab:
        data1.loc[data1["Hour"].str.startswith(x), "Hour"] = 'Unknown'
notime(letras)

datavalue = data1[["Hour"]].stack().value_counts().index.tolist()
data1["Hour"].value_counts()




334

Unknown    2675
11h         228
16h         224
14h         222
15h         218
12h         187
13h         182
17h         175
10h         160
18h         113
09h         109
08h          84
07h          71
19h          49
06h          30
20h          27
03h           8
05h           7
23h           6
04h           5
02h           4
21h           4
01h           3
22h           3
00h           1
Name: Hour, dtype: int64

In [1617]:
# Corregimos el espacio que había delante de TONGA
data1['Country'] = data1['Country'].str.upper()
data1.loc[data1['Country'].str.startswith(" T"), 'Country'] = 'TONGA'
datavalue = data1[["Country"]].stack().value_counts().index.tolist()
display(len(datavalue))
data1['Country'].value_counts()


129

USA                     1958
AUSTRALIA               1094
SOUTH AFRICA             527
PAPUA NEW GUINEA         113
BRAZIL                    93
NEW ZEALAND               89
MEXICO                    63
BAHAMAS                   60
ITALY                     51
PHILIPPINES               38
REUNION                   38
MOZAMBIQUE                38
NEW CALEDONIA             36
FIJI                      33
SPAIN                     29
JAPAN                     26
CROATIA                   25
CUBA                      24
EGYPT                     23
IRAN                      20
FRENCH POLYNESIA          18
PANAMA                    17
TONGA                     14
INDONESIA                 14
SOLOMON ISLANDS           13
JAMAICA                   13
VANUATU                   11
MARSHALL ISLANDS           9
ENGLAND                    9
UNITED KINGDOM             9
                        ... 
THE BALKANS                1
ANTIGUA                    1
CYPRUS                     1
IRAN / IRAQ   

In [1618]:
#Cambiamos algunos de los valores de la columna Sex
datavalue = data1[["Sex"]].stack().value_counts().index.tolist()
display(datavalue)

def nosex(*lab):
    for x in lab:
        data1.loc[data1['Sex'].str.startswith(x), 'Sex'] = 'Unknown'
nosex('.', 'lli', 'N')

def msex(*lab):
    for x in lab:
        data1.loc[data1['Sex'].str.startswith(x), 'Sex'] = 'M'
msex('M ')

datavalue = data1[["Sex"]].stack().value_counts().index.tolist()

data1["Sex"].value_counts()

['M', 'F', 'Unknown', 'M ', '.', 'lli', 'N']

M          3847
F           524
Unknown     424
Name: Sex, dtype: int64

In [1619]:
def edad(lab):
    for x in lab:
        data1.loc[data1["Age"].str.startswith(x), "Age"] = str(x)
    ed = []
    for y in range(10,90):
        ed.append(str(y))
edad(ed)

#for (i, elem) in enumerate(data1["Age"]):
    #print(i, elem)
    
#No he conseguido automatizar que me sacara el index, ni cambiar los valores de esos index por Unknown y los he tenido que extraer a mano y eliminar
ind = [386,695,1197,1262,1327,1411,1490,1491,1564,1614,2050,2548,2564,2677,2911,2968,2984,3030,3134,3271,3503,3515,3652,3804,3853,3894,4012,4041,4129,4141,4379,4460,4688,4777]
data1 = data1.drop(data1.index[ind])

def no(*lab):
    for x in lab:
        data1.loc[data1["Age"].str.contains(x), "Age"] = "Unknown"
no('young','A.M.','teen','Teen')

datavalue = data1[["Age"]].stack().value_counts().index.tolist()
display(len(datavalue))

data1["Age"].value_counts()

78

Unknown    1778
17          139
18          137
20          128
16          128
19          127
15          123
21          106
22           98
25           98
24           91
14           89
13           83
23           77
26           74
29           73
27           73
30           73
28           70
12           68
32           59
35           58
31           51
10           51
40           47
38           42
34           41
43           39
36           38
37           35
           ... 
54           14
57           14
6            13
53           12
60           11
61           11
59           10
58           10
56            9
63            9
69            8
5             7
62            7
70            6
68            6
64            4
65            4
71            3
74            2
75            2
78            2
66            2
77            2
73            1
67            1
86            1
84            1
3             1
87            1
81            1
Name: Age, Length: 78, d

In [1620]:
datavalue = data1[["Activity"]].stack().value_counts().index.tolist()
display(len(datavalue))

#Vamos a reducir de 1098 actividades a categorizarlas en 21 categorías
data1['Activity'] = data1['Activity'].str.lower()

def fishing(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Fishing'
fishing('fish','lobs','catch','wade','harp','fly f','spear','hunt')

def surfing(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Surfing'
surfing('surf','"riding')

def swiming(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Swiming'
swiming('swim','tread','float','bath','"swim"','wadi','"swim')

def kayaking(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Kayaking'
kayaking('kaya','canoe')

def diving(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Diving'
diving('scub','free','div','skin','pearl')

def kite(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Kite'
kite('kite')

def snorkeling(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Snorkeling'
snorkeling('snor')

def sailing(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Sailing'
sailing('boat','sail')

def air(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Air Disaster'
air('argentin','air','boeing','"flying')

def body(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Bodyboarding'
body('boogie','body')

def play(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Playing'
play('play')

def paddling(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Paddling'
paddling('padd')

def jump(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Jumping'
jump('jump')

def shark(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Shark Fishing'
shark('shark','remov','kill')

def stand(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Standing'
stand('stand')

def unk(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Unknown'
unk('no ','.',' ')

def kite(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Kitesurfing'
kite('kite')

def sea(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Sea Disaster'
sea('sea d','ferry','3','2','1','4','9')

def wind(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Windsurfing'
wind('wind')

def helping(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Helping'
helping('help')

def photo(*lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Photographing'
photo('photo')

lower =['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
def photo(lab):
    for x in lab:
        data1.loc[data1['Activity'].str.startswith(x), 'Activity'] = 'Other'
photo(lower)

datavalue = data1[["Activity"]].stack().value_counts().index.tolist()
data1["Activity"].value_counts()

1112

Swiming          1120
Surfing           972
Fishing           847
Other             838
Diving            373
Bodyboarding      166
Standing          117
Snorkeling         72
Kayaking           57
Paddling           28
Shark Fishing      27
Sailing            26
Playing            22
Sea Disaster       21
Windsurfing        18
Jumping            17
Kite               15
Air Disaster       14
Unknown             5
Helping             3
Photographing       3
Name: Activity, dtype: int64

In [1621]:
# Unificamos los valores de la columna Fatal a Y/N/UNKNOWN
datavalue = data1[["Fatal (Y/N)"]].stack().value_counts().index.tolist()

def N(*lab):
    for x in lab:
        data1.loc[data1['Fatal (Y/N)'].str.startswith(x), 'Fatal (Y/N)'] = 'N'
N(' N','N ')

def Unkf(*lab):
    for x in lab:
        data1.loc[data1['Fatal (Y/N)'].str.startswith(x), 'Fatal (Y/N)'] = 'UNKNOWN'
Unkf('F')

datavalue = data1[["Fatal (Y/N)"]].stack().value_counts().index.tolist()

data1["Fatal (Y/N)"].value_counts()




N          3698
Y           997
UNKNOWN      66
Name: Fatal (Y/N), dtype: int64

In [1622]:
data1

,Year,Month,Hour,Country,Area,Location,Sex,Age,Activity,Fatal (Y/N)
0,2016,Unknown,13h,USA,Florida,"New Smyrna Beach, Volusia County",M,16,Surfing,N
1,2016,Unknown,11h,USA,Florida,"New Smyrna Beach, Volusia County",M,36,Surfing,N
2,2016,Unknown,10h,USA,Florida,"New Smyrna Beach, Volusia County",M,43,Surfing,N
3,2016,Unknown,Unknown,AUSTRALIA,Victoria,Thirteenth Beach,M,Unknown,Surfing,N
4,2016,Unknown,Unknown,AUSTRALIA,Victoria,Bells Beach,M,Unknown,Surfing,N
5,2016,Unknown,Unknown,AUSTRALIA,Western Australia,Bunbury,Unknown,Unknown,Fishing,N
6,2016,Unknown,15h,USA,Florida,"Ponte Vedra, St. Johns County",M,60,Swiming,N
7,2016,Unknown,14h,USA,Hawaii,"Makaha, Oahu",F,51,Swiming,N
8,2016,Unknown,15h,NEW CALEDONIA,North Province,Koumac,M,50,Kite,Y
9,2016,Unknown,Unknown,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",F,12,Bodyboarding,N
